In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read the dataset into a pandas dataframe
dfyellow = pd.read_csv("rawdata/yellow_tripdata_2020-03.csv", \
            dtype={'PULocationID': int, 'total_amount': float}, \
            usecols=['PULocationID', 'total_amount'])

dfgreen = pd.read_csv("rawdata/green_tripdata_2020-03.csv", \
            dtype={'PULocationID': int, 'total_amount': float}, \
            usecols=['PULocationID', 'total_amount'])

In [3]:
dfyellow

,PULocationID,total_amount
0,88,27.80
1,193,3.80
2,246,11.75
3,151,10.56
4,79,24.35
...,...,...
3007287,137,33.77
3007288,137,41.27
3007289,137,39.90
3007290,137,22.87


In [4]:
dfzone = pd.read_csv('taxi_data/taxi+_zone_lookup.csv')

In [5]:
dfzone

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,NV,NaN


LocationID 264 and 265 can be dropped as they also don't have a corresponding polygon in the shapefile

In [6]:
taxi_data = dfyellow.groupby(['PULocationID']).sum().reindex(range(1, len(dfzone)+1)).fillna(0)
green_taxi_data = dfgreen.groupby(['PULocationID']).sum().reindex(range(1, len(dfzone)+1)).fillna(0)

In [7]:
def myLog(x):
    return np.log(x) if (x > 0) else 0

#taxi_data['PULocationID'] = taxi_data.index
taxi_data.reset_index(level=0, inplace=True)
taxi_data["log_total_amount"] = np.array(list(map(myLog, taxi_data["total_amount"])))

green_taxi_data.reset_index(level=0, inplace=True)
green_taxi_data["log_total_amount"] = np.array(list(map(myLog, green_taxi_data["total_amount"])))

In [8]:
taxi_data["green_total_amount"] = green_taxi_data["total_amount"]
taxi_data["green_log_total_amount"] = green_taxi_data["log_total_amount"]
taxi_data.rename(columns={
                    "total_amount": "yellow_total_amount",
                    "log_total_amount": "yellow_log_total_amount"
                }, inplace=True)
taxi_data["Borough"] = dfzone["Borough"]
taxi_data["Zone"] = dfzone["Zone"]
taxi_data["service_zone"] = dfzone["service_zone"]

In [9]:
taxi_data

,PULocationID,yellow_total_amount,yellow_log_total_amount,green_total_amount,green_log_total_amount,Borough,Zone,service_zone
0,1,31304.97,10.351532,369.26,5.911501,EWR,Newark Airport,EWR
1,2,9.80,2.282382,6.59,1.885553,Queens,Jamaica Bay,Boro Zone
2,3,2697.85,7.900210,4478.20,8.406976,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,96227.39,11.474469,855.91,6.752165,Manhattan,Alphabet City,Yellow Zone
4,5,533.07,6.278653,0.00,0.000000,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...,...,...,...,...
260,261,353550.28,12.775781,826.71,6.717454,Manhattan,World Trade Center,Yellow Zone
261,262,688824.37,13.442742,470.15,6.153052,Manhattan,Yorkville East,Yellow Zone
262,263,979706.53,13.795008,3002.55,8.007217,Manhattan,Yorkville West,Yellow Zone
263,264,408870.17,12.921153,3969.16,8.286310,Unknown,NV,NaN


In [10]:
taxi_data.to_csv("taxi_data.csv", encoding='utf-8', index=False)